In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, ServiceContext, get_response_synthesizer
from llama_index.readers.file import PyMuPDFReader, UnstructuredReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SentenceTransformerRerank
import ocrmypdf
import faiss
import os
from PyPDF2 import PdfReader, PdfWriter
from pathlib import Path

import os

os.environ["PATH"] += os.pathsep + r"C:\Program Files\gs\gs10.03.0\bin"

from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama

pdf_path = "SP63.pdf"
poppler_path = r"C:\Users\homer\OneDrive\Desktop\Engineering RAG\poppler\Library\bin"

#=== 1) Text loading from file ===
txt_loader = PyMuPDFReader()
txt_docs = txt_loader.load(file_path=pdf_path)

# === 2) OCR for blank pages ===
txt_collection = []
for i, page in enumerate(txt_docs):

    #----- Add text from an image to a text processed correctly -----
    if len(page.text.strip()) < 100:
        # print(f"Page {i}:len {len(page)}")

        temp_file = f"temp_page_{i + 1}.pdf"
        temp_ocr_file = f"ocr_temp_page_{i + 1}.pdf"
        with open(temp_file, "wb") as f:
            reader = PdfReader(pdf_path)
            writer = PdfWriter()
            writer.add_page(reader.pages[i])
            writer.write(f)

        ocrmypdf.ocr(
            temp_file,
            temp_ocr_file,
            language="rus",
            force_ocr=True
        ) 
        ocr_text = txt_loader.load(file_path=temp_ocr_file)
        txt_collection.append(ocr_text[0])
        Path(temp_file).unlink()
        Path(temp_ocr_file).unlink()
    else:
        txt_collection.append(page)
txt_collection

LLM is explicitly disabled. Using MockLLM.


Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

[Document(id_='f0efcda2-2dcd-4525-b731-e32691b45ef8', embedding=None, metadata={'total_pages': 151, 'file_path': 'SP63.pdf', 'source': '1'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='СП 63.13330.2012\n5\n4 Общие требования к бетонным и железобетонным конструкциям\n4.1 Бетонные и железобетонные конструкции всех типов должны удовлетворять\nтребованиям:\nпо безопасности;\nпо эксплуатационной пригодности;\nпо долговечности,\nа также дополнительным требованиям, указанным в задании на проектирование.\n4.2 Для удовлетворения требований по безопасности конструкции должны иметь\nтакие начальные характеристики, чтобы при различных расчетных воздействиях в\nпроцессе строительства и эксплуатации зданий и сооружений были исключены\nразрушения любого характера или нарушения эксплуатационной пригодности,\nсвязанные с причинением вреда жизни

In [ ]:
# === 3) Chancing ===
splitter = SentenceSplitter(chunk_size=500, chunk_overlap=50)
# chunks = splitter.split_documents(txt_collection) # It's not necessary maybe
chunks = splitter.get_nodes_from_documents(txt_collection)

# === 4) Embeddings and index ===
embed_model = OllamaEmbedding(model_name="llama3")
dimension = 768
faiss_index = faiss.IndexFlatL2(dimension)
faiss_store = FaissVectorStore(faiss_index=faiss_index)

index = VectorStoreIndex(
    chunks, 
    vector_store=faiss_store, 
    embed_model=embed_model
)


In [34]:
text = []
for i in chunks:
    text.append(i.text)

c = ''
for j in text:
    if "Бетонные элементы применяют" in j:
        c = j

c

'7 Бетонные конструкции\nКонструкции рассматривают как бетонные, если их прочность обеспечена одним\nтолько бетоном.\nБетонные элементы применяют:\nа) преимущественно на сжатие при расположении продольной сжимающей силы в\nпределах поперечного сечения элемента;\nб) в отдельных случаях в конструкциях, работающих на сжатие при\nрасположении продольной сжимающей силы за пределами поперечного сечения\nэлемента, а также в изгибаемых конструкциях, когда их разрушение не представляет\nнепосредственной опасности для жизни людей и сохранности оборудования.\nКонструкции с арматурой, площадь сечения которой меньше минимально\nдопустимой по конструктивным требованиям 10.3, рассматривают как бетонные.\n7.1 Расчет бетонных элементов по прочности\n7.1.1 Бетонные элементы рассчитывают по прочности на действие продольных\nсжимающих сил, изгибающих моментов и поперечных сил, а также на местное сжатие.\n7.1.2 Расчет по прочности бетонных элементов при действии продольной\nсжимающей силы (внецентренное сж

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer
from llama_index.core.schema import QueryBundle

# === 5) retriever ===
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10, 
)

# === 6) Postprocessors - filter aor reranker ===
postprocessors = [
    # SimilarityPostprocessor(similarity_cutoff=0.65),
    SentenceTransformerRerank(
        top_n=3,
        model="cross-encoder/ms-marco-MiniLM-L-6-v2"
    )
]

# === 7) Synthesis of the response from LLM ===
response_synthesizer = get_response_synthesizer()

# === 7.1) Secure query function ===
def safe_query(query: str) -> str:
    query_bundle = QueryBundle(query)
    
    #--- Extracting candidates ---
    nodes = retriever.retrieve(query_bundle)
    
    #--- Postprocessing ---
    for postprocessor in postprocessors:
        nodes = postprocessor.postprocess_nodes(nodes, query_bundle=query_bundle)

    #--- Relevance check ---
    if not nodes:
        return "❌ По данному запросу не найдена релевантной информация в документе."

    #--- Genarate response of LLM ---
    response = response_synthesizer.synthesize(query_bundle, nodes)
    return str(response)

#=== 8) Answer  ===
query = "Когда применяют бетонные элементы?"
response = safe_query(query)
print("🤖 Ответ:", response)

🤖 Ответ: Context information is below.
---------------------
total_pages: 151
file_path: SP63.pdf
source: 106

При определении кривизны от непродолжительного действия нагрузки в расчете
используют диаграммы кратковременного деформирования сжатого и растянутого
бетона, а при определении кривизны от продолжительного действия нагрузки –
диаграммы длительного деформирования бетона с расчетными характеристиками для
предельных состояний второй группы.

total_pages: 151
file_path: SP63.pdf
source: 4

5.1.5 Расчеты бетонных и железобетонных конструкций производят на действие
изгибающих моментов, продольных сил, поперечных сил и крутящих моментов, а
также на местное действие нагрузки.

total_pages: 151
file_path: SP63.pdf
source: 8

СП 63.13330.2012
12
бетонной полосы, находящейся под воздействием сжимающих усилий вдоль полосы и
растягивающих усилий от поперечной арматуры, пересекающей наклонную полосу.
---------------------
Given the context information and not prior knowledge, answer the quer

Вариант без защиты

In [39]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor, SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import get_response_synthesizer

# === retriever ===
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10, 
)

# === Postprocessors - filter aor reranker ===
postprocessors = [
    SimilarityPostprocessor(similarity_cutoff=0.7),
    # SentenceTransformerRerank(
    #     top_n=5,
    #     model="cross-encoder/ms-marco-MiniLM-L-6-v2"
    # )
]

response_synthesizer = get_response_synthesizer()

# === 4. Query Engine ===
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=postprocessors,
    response_synthesizer=response_synthesizer,
)


query = "Когда применяют бетонные элементы?"
response = query_engine.query(query)
print(response)

Context information is below.
---------------------
total_pages: 151
file_path: SP63.pdf
source: 11

5.5.5 Значения предельно допустимых деформаций принимают в соответствии с
указаниями 8.2.20. При действии постоянных и временных длительных и
кратковременных нагрузок прогиб железобетонных элементов во всех случаях не
должен превышать 1/150 пролета и 1/75 вылета консоли.
6 Материалы для бетонных и железобетонных конструкций
6.1 Бетон
6.1.1 Для бетонных и железобетонных конструкций, проектируемых в
соответствии с требованиями настоящего свода правил, следует предусматривать
конструкционные бетоны:
тяжелый средней плотности от 2200 до 2500 кг/м3 включительно;
мелкозернистый средней плотности от 1800 до 2200 кг/м3;
легкий;
ячеистый;
напрягающий.
6.1.2 При проектировании бетонных и железобетонных сооружений в
соответствии с требованиями, предъявленными к конкретным конструкциям, должны
быть установлены вид бетона и его нормируемые показатели качества (ГОСТ 25192,
ГОСТ 4.212), контролируемые